In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-01-05 08:34:24--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-01-05 08:34:25 (86.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback, plot_loss_curves,compare_historys,unzip_data

## Get text Dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2022-01-05 08:34:27--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2022-01-05 08:34:27 (40.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [6]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)

In [7]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
len(train_df), len(test_df)

(7613, 3263)

In [11]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target:{target}","(real diasster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")
  # print(row)
  # break

Target:1 (real diasster)
Text:
*looks outside at the windstorm* niiiice

---

Target:0 (not real disaster)
Text:
Fylde Building set to be flattened: One of PrestonÛªs city centre iconic buildings is disappearing from the skyline. http://t.co/PdKHBdG9hO

---

Target:1 (real diasster)
Text:
Estimated damage and fatalities of an Hiroshima-sized atomic bomb dropped on your hometown - http://t.co/BSrERJbY0I  #Hiroshima70

---

Target:0 (not real disaster)
Text:
@Karnythia my niece is gaining the ability to stand. I'm getting prepared for toddler apocalypse Armageddon

---

Target:1 (real diasster)
Text:
The thunder shook my house woke my sister and made car alarms go off ????

---



In [12]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers

In [13]:
#use train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [14]:
from re import split
text_vectorizatin = TextVectorization(
    max_tokens=None,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None,
    output_mode="int",
    output_sequence_length=None,
    pad_to_max_tokens=False
)

In [15]:
max_vocab_length= 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)

In [16]:
# fit the text vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
random_sentence

'@ModelBubbles GM! I pray any attack of the enemy 2 derail ur destiny is blocked by the Lord &amp; that He floods ur life w/heavenly Blessings'

In [19]:
text_vectorizer([random_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 1671,    8,  997,  202,  113,    6,    2, 2127,   70,  465,
         638, 2140,    9, 1321]])>

In [20]:
text_vectorizer.get_vocabulary()[:5],

(['', '[UNK]', 'the', 'a', 'in'],)

# Make embedding layer

In [21]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [22]:
random_sentence = random.choice(train_sentences)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed.shape

TensorShape([1, 15, 128])

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


#Create tokenization and modeling pipeline

model_0 = Pipeline([
    ("cfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('cfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
#Evaluate baseline model

base_line_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieved an accuracy of: {base_line_score*100:.2f}%")

Our baseline model achieved an accuracy of: 79.27%


In [25]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support 

def calculate_results(y_true, y_pred):

  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average="weighted")

  model_results = {
      "accuracy": model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_results

In [27]:
base_line_results = calculate_results(val_labels, baseline_preds)
base_line_results 

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

## Feed forward neural network model

In [28]:
#Create tensorboard callback

from helper_functions import create_tensorboard_callback

# create a directory to save tensorflow logs

SAVE_DIR = "model_logs"

In [29]:
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")

In [30]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [31]:
model_1.compile(loss="binary_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

In [32]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_1")])


Saving TensorBoard log files to: model_logs/model_1/20220105-083429
Epoch 1/5
215/215 [==============================] - 4s 14ms/step - loss: 0.6090 - accuracy: 0.6920 - val_loss: 0.5335 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4387 - accuracy: 0.8205 - val_loss: 0.4672 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3454 - accuracy: 0.8594 - val_loss: 0.4556 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2824 - accuracy: 0.8913 - val_loss: 0.4625 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2358 - accuracy: 0.9142 - val_loss: 0.4788 - val_accuracy: 0.7874


In [33]:
model_1_pred_probs = model_1.predict(val_sentences)

In [34]:
model_1_pred_probs[:20]

array([[0.42051363],
       [0.7399354 ],
       [0.9979789 ],
       [0.11043906],
       [0.09903115],
       [0.935166  ],
       [0.9102471 ],
       [0.9950878 ],
       [0.9649333 ],
       [0.25981033],
       [0.1257019 ],
       [0.6352414 ],
       [0.06433392],
       [0.21471313],
       [0.00541681],
       [0.14073873],
       [0.03029081],
       [0.08279616],
       [0.24969274],
       [0.54163474]], dtype=float32)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [36]:
model_1_results = calculate_results(val_labels,model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'f1': 0.7848342804717284,
 'precision': 0.7911061029933195,
 'recall': 0.7874015748031497}

# Recurrent neural network

### LSTM(long short term memory)

In [37]:
#create a  LSTM

from tensorflow.keras import layers


inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.LSTM(128,return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(256)(x)
# print(x.shape)
x = layers.Dense(64,activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")

In [38]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 128)           131584    
                                                                 
 lstm_1 (LSTM)               (None, 256)               394240    
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                      

In [39]:
model_2.compile(loss="binary_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

In [40]:
history_2_history = model_2.fit(train_sentences,
                                    train_labels,
                                    epochs=5,
                                    validation_data=(val_sentences,val_labels),
                                    callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2")])

Saving TensorBoard log files to: model_logs/model_2/20220105-083451
Epoch 1/5
215/215 [==============================] - 25s 96ms/step - loss: 0.2101 - accuracy: 0.9240 - val_loss: 0.5358 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 20s 91ms/step - loss: 0.1582 - accuracy: 0.9437 - val_loss: 0.6345 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 19s 91ms/step - loss: 0.1281 - accuracy: 0.9529 - val_loss: 0.7432 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 20s 91ms/step - loss: 0.1125 - accuracy: 0.9572 - val_loss: 0.8093 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 20s 91ms/step - loss: 0.0895 - accuracy: 0.9644 - val_loss: 0.8323 - val_accuracy: 0.7756


In [41]:
model_2_pred_probs = model_2.predict(val_sentences)

In [42]:
model_1_pred_probs[:10]

array([[0.42051363],
       [0.7399354 ],
       [0.9979789 ],
       [0.11043906],
       [0.09903115],
       [0.935166  ],
       [0.9102471 ],
       [0.9950878 ],
       [0.9649333 ],
       [0.25981033]], dtype=float32)

In [43]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [44]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_2_results = calculate_results(val_labels,model_1_preds)
model_2_results

{'accuracy': 78.74015748031496,
 'f1': 0.7848342804717284,
 'precision': 0.7911061029933195,
 'recall': 0.7874015748031497}

# GRU

In [46]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64,return_sequences=True)(x)
# x = layers.LSTM(64,return_sequences=True)(x)
x = layers.GRU(64)(x)
# x = layers.Dense(64,activation="relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name="model_3_GRU")

In [47]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [48]:
model_3.compile(loss="binary_crossentropy",
                                  optimizer="Adam",
                                  metrics=["accuracy"])

In [49]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3")])

Saving TensorBoard log files to: model_logs/model_3/20220105-083636
Epoch 1/5
215/215 [==============================] - 8s 25ms/step - loss: 0.1633 - accuracy: 0.9349 - val_loss: 0.7036 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0839 - accuracy: 0.9691 - val_loss: 0.8149 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0717 - accuracy: 0.9711 - val_loss: 1.0182 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0588 - accuracy: 0.9753 - val_loss: 1.2594 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0507 - accuracy: 0.9769 - val_loss: 1.3016 - val_accuracy: 0.7690


In [50]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[8.0209970e-04],
       [7.2259665e-01],
       [9.9988842e-01],
       [9.3594581e-02],
       [3.0396064e-05],
       [9.9938619e-01],
       [6.5317011e-01],
       [9.9997306e-01],
       [9.9992055e-01],
       [4.9589834e-01]], dtype=float32)

In [51]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [52]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 76.9028871391076,
 'f1': 0.766385677090764,
 'precision': 0.7717390408616506,
 'recall': 0.7690288713910761}

## Bidirectional RNN

In [53]:
# Build a bidirectional RNN

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x) 
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [54]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [55]:
#compile model
model_4.compile(loss="binary_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

In [56]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4")])

Saving TensorBoard log files to: model_logs/model_4/20220105-083719
Epoch 1/5
215/215 [==============================] - 11s 36ms/step - loss: 0.1104 - accuracy: 0.9638 - val_loss: 0.8764 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 7s 31ms/step - loss: 0.0528 - accuracy: 0.9766 - val_loss: 1.2945 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 7s 31ms/step - loss: 0.0454 - accuracy: 0.9794 - val_loss: 1.2302 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 7s 32ms/step - loss: 0.0430 - accuracy: 0.9799 - val_loss: 1.3554 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 7s 31ms/step - loss: 0.0424 - accuracy: 0.9804 - val_loss: 1.4509 - val_accuracy: 0.7651


In [57]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[6.8715513e-03],
       [4.8154011e-01],
       [9.9999553e-01],
       [2.1295720e-01],
       [4.7361147e-05],
       [9.9995220e-01],
       [9.9442923e-01],
       [9.9999785e-01],
       [9.9999654e-01],
       [9.9875909e-01]], dtype=float32)

In [58]:
model_4_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [59]:
model_4_results = calculate_results(val_labels, model_1_preds)
model_4_results

{'accuracy': 78.74015748031496,
 'f1': 0.7848342804717284,
 'precision': 0.7911061029933195,
 'recall': 0.7874015748031497}

# Conv1D

In [65]:
#create conv1d sequence model

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(32,5,activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs)

In [66]:
model_5.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_average_pooling1d_2   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_5 (Dense)             (None, 1)                 33    

In [67]:
model_5.compile(loss="binary_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

In [68]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_5")])

Saving TensorBoard log files to: model_logs/model_5/20220105-090200
Epoch 1/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1787 - accuracy: 0.9441 - val_loss: 0.8142 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0957 - accuracy: 0.9602 - val_loss: 0.9707 - val_accuracy: 0.7546
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0733 - accuracy: 0.9711 - val_loss: 1.1709 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0622 - accuracy: 0.9734 - val_loss: 1.2520 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0549 - accuracy: 0.9762 - val_loss: 1.3825 - val_accuracy: 0.7546


In [69]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[7.8356081e-01],
       [9.0178806e-01],
       [9.9999547e-01],
       [1.0122666e-01],
       [7.7078845e-08],
       [9.9505460e-01],
       [9.8233473e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.0086567e-01]], dtype=float32)

In [70]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [72]:
model_5_results = calculate_results(val_labels,model_5_preds)
model_5_results

{'accuracy': 75.45931758530183,
 'f1': 0.7540737922598122,
 'precision': 0.7541438250099668,
 'recall': 0.7545931758530183}

## Transfer learning in NLP

In [74]:
import tensorflow_hub as hub

In [81]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
                    "hello this is me",
                    "I am testing this embbeding using universal"
])
embeddings.shape

TensorShape([2, 512])

In [84]:
# Create a model using sequential API
sentence_encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                  trainable=False,
                                  name="USE")

In [100]:
inputs = layers.Input(shape=[], dtype=tf.string)
# print(inputs.shape)
x = sentence_encoder(inputs)
x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_6 = tf.keras.Model(inputs,outputs,name="model_6")
model_6.compile(loss="binary_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

In [101]:
model_6.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None,)]                 0         
                                                                 
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_13 (Dense)            (None, 64)                32832     
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [102]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_6")])

Saving TensorBoard log files to: model_logs/model_6/20220105-095622
Epoch 1/5
215/215 [==============================] - 4s 12ms/step - loss: 0.5064 - accuracy: 0.7816 - val_loss: 0.4499 - val_accuracy: 0.7966
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4145 - accuracy: 0.8178 - val_loss: 0.4394 - val_accuracy: 0.8058
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4010 - accuracy: 0.8209 - val_loss: 0.4349 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3946 - accuracy: 0.8232 - val_loss: 0.4300 - val_accuracy: 0.8136
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3865 - accuracy: 0.8320 - val_loss: 0.4292 - val_accuracy: 0.8163


In [103]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.20391497],
       [0.8248297 ],
       [0.9893667 ],
       [0.18031642],
       [0.715761  ],
       [0.77400064],
       [0.98232394],
       [0.9778825 ],
       [0.9434148 ],
       [0.09960118]], dtype=float32)

In [104]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [105]:
model_6_results = calculate_results(val_labels,model_6_preds)
model_6_results

{'accuracy': 81.62729658792651,
 'f1': 0.8151782870321896,
 'precision': 0.8174065945777478,
 'recall': 0.8162729658792651}

## Get 10% data for training model 7

In [127]:
# train_10_percent = train_df_shuffled[["text","target"]].sample(frac=0.1, random_state=42)
# train_sentences_10 = train_10_percent["text"].to_list()
# train_labels_10 = train_10_percent["target"].to_list()
# len(train_sentences_10),len(train_labels_10)

train_10_percent_split = int(.1*len(train_sentences))
train_10_percent_split

train_sentences_10 = train_sentences[:train_10_percent_split]
train_labels_10 = train_labels[:train_10_percent_split]

In [128]:
pd.DataFrame(train_labels_10).value_counts()

0    406
1    279
dtype: int64

In [129]:
inputs = layers.Input(shape=[],dtype=tf.string)
x = sentence_encoder(inputs)
x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_7 = tf.keras.Model(inputs,outputs,name="model_7")
model_7.compile(loss="binary_crossentropy",
                optimizer="Adam",
                metrics=["accuracy"])

In [130]:
model_7.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None,)]                 0         
                                                                 
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_21 (Dense)            (None, 64)                32832     
                                                                 
 dense_22 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [131]:
model_7_history = model_7.fit(train_sentences_10,
                              train_labels_10,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_7")])

Saving TensorBoard log files to: model_logs/model_7/20220105-111435
Epoch 1/5
22/22 [==============================] - 2s 48ms/step - loss: 0.6696 - accuracy: 0.6219 - val_loss: 0.6504 - val_accuracy: 0.6772
Epoch 2/5
22/22 [==============================] - 1s 26ms/step - loss: 0.6040 - accuracy: 0.7825 - val_loss: 0.5947 - val_accuracy: 0.7717
Epoch 3/5
22/22 [==============================] - 1s 25ms/step - loss: 0.5306 - accuracy: 0.8102 - val_loss: 0.5392 - val_accuracy: 0.7756
Epoch 4/5
22/22 [==============================] - 0s 19ms/step - loss: 0.4691 - accuracy: 0.8175 - val_loss: 0.5040 - val_accuracy: 0.7808
Epoch 5/5
22/22 [==============================] - 1s 25ms/step - loss: 0.4257 - accuracy: 0.8336 - val_loss: 0.4902 - val_accuracy: 0.7808


In [132]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

array([[0.2136895 ],
       [0.6149656 ],
       [0.9134537 ],
       [0.36359787],
       [0.55742747],
       [0.67616665],
       [0.8860128 ],
       [0.785069  ],
       [0.83993995],
       [0.15375909]], dtype=float32)

In [133]:
model_7_preds =  tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [134]:
model_7_results = calculate_results(val_labels,model_7_preds)
model_7_results

{'accuracy': 78.08398950131233,
 'f1': 0.7793746882098481,
 'precision': 0.7816731653312017,
 'recall': 0.7808398950131233}